In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from glob import glob
from tqdm import tqdm
import xarray as xr
import os, sys
import scipy
import pandas as pd
from scipy.ndimage import gaussian_filter
from sklearn.decomposition import PCA
from scipy import stats

from scipy import interpolate

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

from allenAnalysis.utils import *

%matplotlib ipympl

In [25]:
from allenAnalysis.icsd import StandardCSD, DeltaiCSD, StepiCSD

In [3]:
import matplotlib

font = {'size'   : 12}

matplotlib.rc('font', **font)

In [4]:
matplotlib.rcParams['savefig.pad_inches'] = 0

In [5]:
typesL = np.load('EEGs.npy',allow_pickle=True).item()
totalL = np.load('total.npy')
# totalL = totalL[0:len(totalL):10]

In [6]:
lfpIdx = np.arange(51)
sphere_fullDensity_idx = np.arange(51,102)
disk_fullDensity_idx = np.arange(102,153)
sphere_halfDensity_idx = np.arange(153,179)
disk_halfDensity_idx = np.arange(179,205)
sphere_quarterDensity_idx = np.arange(205,218)
disk_quarterDensity_idx = np.arange(218,231)


In [7]:
totalLFP = totalL[:,lfpIdx]
objective_sphere_full = totalL[:,sphere_fullDensity_idx]
objective_disk_full = totalL[:,disk_fullDensity_idx]
objective_sphere_half = totalL[:,sphere_halfDensity_idx]
objective_disk_half = totalL[:,disk_halfDensity_idx]
objective_sphere_quarter = totalL[:,sphere_quarterDensity_idx]
objective_disk_quarter = totalL[:,disk_quarterDensity_idx]

In [8]:
import h5py
f = h5py.File('/gpfs/bbp.cscs.ch/project/proj68/scratch/tharayil/newCoeffs/b3122c45-e2d9-4205-821c-88f28791dc69/0/neuropixels/coeffsneuropixels.h5')


In [9]:
electrodePos = []
for e in np.arange(51):
    electrodePos.append(f['electrodes']['Neuropixels-384_'+str(e)]['location'][:])
    
electrodePos = np.array(electrodePos)

In [10]:
np.linalg.norm(electrodePos[1]-electrodePos[0])

40.00061443278048

In [11]:
layers = []

for i in range(51):
    
    l = str(f['electrodes']['Neuropixels-384_'+str(i)]['layer'][()])
    
    if l == "b'Outside'":
        
        layers.append('Outside')
        
    else:
    
        l = l.split('L')[-1]

        layers.append(int(l[0]))

In [12]:
f.close()

In [13]:
breaks = np.where(np.diff(layers)!=0)[0]

In [14]:
meanLfp = np.mean(totalL,axis=0)

In [15]:
totalL -= meanLfp

In [16]:
import bluepysnap as bp
s = bp.Simulation('0/simulation_config.json')
types = np.sort(list(s.circuit.nodes.property_values('mtype')))

In [17]:
byLayer = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer[layer] += typesL[t][:,:52]-np.mean(typesL[t][:,:52],axis=0)
            except:
                byLayer[layer] = typesL[t][:,:52]-np.mean(typesL[t][:,:52],axis=0)
                

In [18]:
byLayer_objective_sphere_full = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer_objective_sphere_full[layer] += typesL[t][:,sphere_fullDensity_idx]-np.mean(typesL[t][:,sphere_fullDensity_idx],axis=0)
            except:
                byLayer_objective_sphere_full[layer] = typesL[t][:,sphere_fullDensity_idx]-np.mean(typesL[t][:,sphere_fullDensity_idx],axis=0)
                

byLayer_objective_sphere_half = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer_objective_sphere_half[layer] += typesL[t][:,sphere_halfDensity_idx]-np.mean(typesL[t][:,sphere_halfDensity_idx],axis=0)
            except:
                byLayer_objective_sphere_half[layer] = typesL[t][:,sphere_halfDensity_idx]-np.mean(typesL[t][:,sphere_halfDensity_idx],axis=0)
                

byLayer_objective_sphere_quarter = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer_objective_sphere_quarter[layer] += typesL[t][:,sphere_quarterDensity_idx]-np.mean(typesL[t][:,sphere_quarterDensity_idx],axis=0)
            except:
                byLayer_objective_sphere_quarter[layer] = typesL[t][:,sphere_quarterDensity_idx]-np.mean(typesL[t][:,sphere_quarterDensity_idx],axis=0)
                

In [19]:
byLayer_objective_disk_full = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer_objective_disk_full[layer] += typesL[t][:,disk_fullDensity_idx]-np.mean(typesL[t][:,disk_fullDensity_idx],axis=0)
            except:
                byLayer_objective_disk_full[layer] = typesL[t][:,disk_fullDensity_idx]-np.mean(typesL[t][:,disk_fullDensity_idx],axis=0)
                

byLayer_objective_disk_half = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer_objective_disk_half[layer] += typesL[t][:,disk_halfDensity_idx]-np.mean(typesL[t][:,disk_halfDensity_idx],axis=0)
            except:
                byLayer_objective_disk_half[layer] = typesL[t][:,disk_halfDensity_idx]-np.mean(typesL[t][:,disk_halfDensity_idx],axis=0)
                

byLayer_objective_disk_quarter = {}

for layer in np.arange(1,7):
    
    
    for i, t in enumerate(types):

        if str(layer) in t:

            try:
                byLayer_objective_disk_quarter[layer] += typesL[t][:,disk_quarterDensity_idx]-np.mean(typesL[t][:,disk_quarterDensity_idx],axis=0)
            except:
                byLayer_objective_disk_quarter[layer] = typesL[t][:,disk_quarterDensity_idx]-np.mean(typesL[t][:,disk_quarterDensity_idx],axis=0)
                

In [20]:
time = np.arange(0,3000,1)

In [21]:
def plotFig(axes,csd,breaks,absmax=None):

    for b in breaks:
        ax.plot([0,3000],[b,b],c='k')

    if absmax is None:
        absmax = np.max(np.abs(csd))/2
        plotCB = True
    else:
        plotCB = False
    
    im = ax.imshow(np.array(csd), origin='lower', vmin =-absmax,vmax=absmax, cmap='bwr_r', interpolation='gaussian')
    ax.axis(ax.axis('tight'))

    if plotCB:
        cb = plt.colorbar(im, ax=ax)
        cb.formatter.set_powerlimits((0,0))


    ax.set_yticklabels([])
    ax.set_xticklabels([])

    ax.set_xlim([2000,2050])

    
    
        

In [22]:
def plotFigResting(axes,csd,breaks,absmax=None):

    for b in breaks:
        ax.plot([0,3000],[b,b],c='k')

    if absmax is None:
        absmax = np.max(np.abs(csd))/2
        plotCB = True
    else:
        plotCB = False
    
    im = ax.imshow(np.array(csd), origin='lower', vmin =-absmax,vmax=absmax, cmap='bwr_r', interpolation='gaussian')
    ax.axis(ax.axis('tight'))

    if plotCB:
        cb = plt.colorbar(im, ax=ax)
        cb.formatter.set_powerlimits((0,0))


    ax.set_yticklabels([])
    ax.set_xticklabels([])

    ax.set_xlim([2450,2500])


In [31]:


fig, axes = plt.subplots(1,3,figsize=(9,3))

lfp_data = totalLFP.T * 1E-3 * pq.V        # [mV] -> [V]

z_data = 40*np.arange(0,51) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 200E-6 * pq.m                              # [m]
h = 40E-6 * pq.m                                 # [m]
sigma = 0.376 * pq.S / pq.m                         # [S/m] or [1/(ohm*m)]
sigma_top = 0.376 * pq.S / pq.m                     # [S/m] or [1/(ohm*m)]

# Input dictionaries for each method

step_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'h':h,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}

csd_dict = dict(
    std_csd = StepiCSD(**step_input),
)

for method, csd_obj in csd_dict.items():
    
    ax = axes[0]
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)
    plotFig(ax,csd,breaks)

ax = axes[1]
csd = objective_disk_full.T
plotFig(ax,csd,breaks)
ax.set_title('Objective, \ndisk')

ax = axes[2]
csd = objective_sphere_full.T
plotFig(ax,csd,breaks)
ax.set_title('Objective, \nsphere')

correlations = []
for i in range(3000):
    correlations.append(np.corrcoef(csd_obj.get_csd()[:,i],objective_disk_full.T[:,i])[0,1])

plt.figure()
plt.plot(correlations)

discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ], 
a = [ 2.213 ]


/gpfs/bbp.cscs.ch/home/tharayil/allen/lib/python3.11/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


In [ ]:
from scipy import io
import quantities as pq
import matplotlib.colors as colors

fig, axes = plt.subplots(3,3,figsize=(9,9))

lfp_data = totalLFP.T * 1E-3 * pq.V        # [mV] -> [V]


z_data = np.arange(0,51) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 20E-6 * pq.m                              # [m]
h = 1E-6 * pq.m                                 # [m]
sigma = 0.277 * pq.S / pq.m                         # [S/m] or [1/(ohm*m)]
sigma_top = 0.277 * pq.S / pq.m                     # [S/m] or [1/(ohm*m)]

# Input dictionaries for each method

std_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}
#Create the different CSD-method class instances. We use the class methods
#get_csd() and filter_csd() below to get the raw and spatially filtered
#versions of the current-source density estimates.
csd_dict = dict(
#     delta_icsd = DeltaiCSD(**delta_input),
#     step_icsd = StepiCSD(**step_input),
#     spline_icsd = SplineiCSD(**spline_input),
    std_csd = StandardCSD(**std_input),
)

for method, csd_obj in csd_dict.items():
    
    
    
    ax = axes[0][0]
        
    #plot raw csd estimate
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)

    plotFig(ax,csd,breaks)
    ax.set_title('iCSD')


ax = axes[0][1]
csd = objective_disk_full.T
plotFig(ax,csd,breaks)
ax.set_title('Objective, \ndisk')

ax = axes[0][2]
csd = objective_sphere_full.T
plotFig(ax,csd,breaks)
ax.set_title('Objective, \nsphere')

lfp_data = totalLFP[:,0:-1:2].T * 1E-3 * pq.V        # [mV] -> [V]

z_data = np.arange(0,51,2) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 40E-6 * pq.m                              # [m]


# Input dictionaries for each method

std_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}
#Create the different CSD-method class instances. We use the class methods
#get_csd() and filter_csd() below to get the raw and spatially filtered
#versions of the current-source density estimates.
csd_dict = dict(
#     delta_icsd = DeltaiCSD(**delta_input),
#     step_icsd = StepiCSD(**step_input),
#     spline_icsd = SplineiCSD(**spline_input),
    std_csd = StandardCSD(**std_input),
)

for method, csd_obj in csd_dict.items():
    
    
    
    ax = axes[1][0]
        
    #plot raw csd estimate
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)

    plotFig(ax,csd,breaks/2)


ax = axes[1][1]
csd = objective_disk_half.T
plotFig(ax,csd,breaks/2)

ax = axes[1][2]
csd = objective_sphere_half.T
plotFig(ax,csd,breaks/2)

lfp_data = totalLFP[:,0:-1:4].T * 1E-3 * pq.V        # [mV] -> [V]

z_data = np.arange(0,51,4) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 80E-6 * pq.m                              # [m]


# Input dictionaries for each method

std_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}
#Create the different CSD-method class instances. We use the class methods
#get_csd() and filter_csd() below to get the raw and spatially filtered
#versions of the current-source density estimates.
csd_dict = dict(
#     delta_icsd = DeltaiCSD(**delta_input),
#     step_icsd = StepiCSD(**step_input),
#     spline_icsd = SplineiCSD(**spline_input),
    std_csd = StandardCSD(**std_input),
)

for method, csd_obj in csd_dict.items():
    
    
    
    ax = axes[2][0]
        
    #plot raw csd estimate
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)

    plotFig(ax,csd,breaks/4)


ax = axes[2][1]
csd = objective_disk_quarter.T
plotFig(ax,csd,breaks/4)

ax = axes[2][2]
csd = objective_sphere_quarter.T
plotFig(ax,csd,breaks/4)


plt.tight_layout()
plt.show()




In [ ]:
from scipy import io
import quantities as pq
import matplotlib.colors as colors

fig, axes = plt.subplots(3,3,figsize=(9,9))

lfp_data = totalLFP.T * 1E-3 * pq.V        # [mV] -> [V]


z_data = np.arange(0,51) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 20E-6 * pq.m                              # [m]
h = 1E-6 * pq.m                                 # [m]
sigma = 0.277 * pq.S / pq.m                         # [S/m] or [1/(ohm*m)]
sigma_top = 0.277 * pq.S / pq.m                     # [S/m] or [1/(ohm*m)]

# Input dictionaries for each method

std_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}
#Create the different CSD-method class instances. We use the class methods
#get_csd() and filter_csd() below to get the raw and spatially filtered
#versions of the current-source density estimates.
csd_dict = dict(
#     delta_icsd = DeltaiCSD(**delta_input),
#     step_icsd = StepiCSD(**step_input),
#     spline_icsd = SplineiCSD(**spline_input),
    std_csd = StandardCSD(**std_input),
)

for method, csd_obj in csd_dict.items():
    
    
    
    ax = axes[0][0]
        
    #plot raw csd estimate
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)

    plotFigResting(ax,csd,breaks)
    ax.set_title('iCSD')


ax = axes[0][1]
csd = objective_disk_full.T
plotFigResting(ax,csd,breaks)
ax.set_title('Objective, \ndisk')

ax = axes[0][2]
csd = objective_sphere_full.T
plotFigResting(ax,csd,breaks)
ax.set_title('Objective, \nsphere')

lfp_data = totalLFP[:,0:-1:2].T * 1E-3 * pq.V        # [mV] -> [V]

z_data = np.arange(0,51,2) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 40E-6 * pq.m                              # [m]


# Input dictionaries for each method

std_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}
#Create the different CSD-method class instances. We use the class methods
#get_csd() and filter_csd() below to get the raw and spatially filtered
#versions of the current-source density estimates.
csd_dict = dict(
#     delta_icsd = DeltaiCSD(**delta_input),
#     step_icsd = StepiCSD(**step_input),
#     spline_icsd = SplineiCSD(**spline_input),
    std_csd = StandardCSD(**std_input),
)

for method, csd_obj in csd_dict.items():
    
    
    
    ax = axes[1][0]
        
    #plot raw csd estimate
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)

    plotFigResting(ax,csd,breaks/2)


ax = axes[1][1]
csd = objective_disk_half.T
plotFigResting(ax,csd,breaks/2)

ax = axes[1][2]
csd = objective_sphere_half.T
plotFigResting(ax,csd,breaks/2)

lfp_data = totalLFP[:,0:-1:4].T * 1E-3 * pq.V        # [mV] -> [V]

z_data = np.arange(0,51,4) * 1e-6 * pq.m
#z_data = np.linspace(0, 2000E-6, 50) * pq.m  # [m]
diam = 80E-6 * pq.m                              # [m]


# Input dictionaries for each method

std_input = {
    'lfp' : lfp_data,
    'coord_electrode' : z_data,
    'sigma' : sigma,
    'f_type' : 'gaussian',
    'f_order' : (3, 1),
}
#Create the different CSD-method class instances. We use the class methods
#get_csd() and filter_csd() below to get the raw and spatially filtered
#versions of the current-source density estimates.
csd_dict = dict(
#     delta_icsd = DeltaiCSD(**delta_input),
#     step_icsd = StepiCSD(**step_input),
#     spline_icsd = SplineiCSD(**spline_input),
    std_csd = StandardCSD(**std_input),
)

for method, csd_obj in csd_dict.items():
    
    
    
    ax = axes[2][0]
        
    #plot raw csd estimate
    csd = csd_obj.get_csd()
    csd = csd_obj.filter_csd(csd)

    plotFigResting(ax,csd,breaks/4)


ax = axes[2][1]
csd = objective_disk_quarter.T
plotFigResting(ax,csd,breaks/4)

ax = axes[2][2]
csd = objective_sphere_quarter.T
plotFigResting(ax,csd,breaks/4)


plt.tight_layout()
plt.show()

In [ ]:
lfp_data.shape

In [ ]:
from scipy import io
import quantities as pq
import matplotlib.colors as colors

originalbreaks = breaks

fig, axes = plt.subplots(3,5,figsize=(12,12))

for j in range(3):

    for i in np.arange(2,7):    

        if j == 0:
            lfp_data = byLayer[i].T * 1E-3 * pq.V
            breaks = originalbreaks
            z_data = np.arange(0,51) * 1e-6 * pq.m
        elif j == 1:
            lfp_data = byLayer[i][:,0:-1:2].T * 1E-3 * pq.V
            breaks = originalbreaks/2
            z_data = np.arange(0,51,2) * 1e-6 * pq.m
        else:
            lfp_data = byLayer[i][:,0:-1:4].T * 1E-3 * pq.V
            breaks = originalbreaks/4
            z_data = np.arange(0,51,4) * 1e-6 * pq.m
        
        diam = 20E-6 * pq.m                              # [m]
        h = 1E-6 * pq.m                                 # [m]
        sigma = 0.277 * pq.S / pq.m                         # [S/m] or [1/(ohm*m)]
        sigma_top = 0.277 * pq.S / pq.m                     # [S/m] or [1/(ohm*m)]
    
        # Input dictionaries for each method
    
        std_input = {
            'lfp' : lfp_data,
            'coord_electrode' : z_data,
            'sigma' : sigma,
            'f_type' : 'gaussian',
            'f_order' : (3, 1),
        }
        #Create the different CSD-method class instances. We use the class methods
        #get_csd() and filter_csd() below to get the raw and spatially filtered
        #versions of the current-source density estimates.
        csd_dict = dict(
            std_csd = StandardCSD(**std_input),
        )
    
        for method, csd_obj in csd_dict.items():
    
            #plot raw csd estimate
            csd = csd_obj.get_csd()
            csd = csd_obj.filter_csd(csd)
            
            ax = axes[j][i-2]
            
            plotFig(ax,csd,breaks,2.5e-2)
    

        breaks = originalbreaks
    
        # plt.tight_layout()
        # plt.show()
    


In [ ]:
from scipy import io
import quantities as pq
import matplotlib.colors as colors

originalbreaks = breaks

fig, axes = plt.subplots(3,5,figsize=(12,12))

for j in range(3):

    for i in np.arange(2,7):    

        if j == 0:
            lfp_data = byLayer_objective_disk_full[i].T * 1E-3 * pq.V
            breaks = originalbreaks
            z_data = np.arange(0,51) * 1e-6 * pq.m
            absmax = 6e-1
        elif j == 1:
            lfp_data = byLayer_objective_disk_half[i].T * 1E-3 * pq.V
            breaks = originalbreaks/2
            z_data = np.arange(0,51,2) * 1e-6 * pq.m
            absmax = 6e-1
        else:
            lfp_data = byLayer_objective_disk_quarter[i].T * 1E-3 * pq.V
            breaks = originalbreaks/4
            z_data = np.arange(0,51,4) * 1e-6 * pq.m
        
        diam = 20E-6 * pq.m                              # [m]
        h = 1E-6 * pq.m                                 # [m]
        sigma = 0.277 * pq.S / pq.m                         # [S/m] or [1/(ohm*m)]
        sigma_top = 0.277 * pq.S / pq.m                     # [S/m] or [1/(ohm*m)]
    
        # Input dictionaries for each method
    
        std_input = {
            'lfp' : lfp_data,
            'coord_electrode' : z_data,
            'sigma' : sigma,
            'f_type' : 'gaussian',
            'f_order' : (3, 1),
        }
        #Create the different CSD-method class instances. We use the class methods
        #get_csd() and filter_csd() below to get the raw and spatially filtered
        #versions of the current-source density estimates.
        csd_dict = dict(
            std_csd = StandardCSD(**std_input),
        )
    
        for method, csd_obj in csd_dict.items():
    
            #plot raw csd estimate
            csd = lfp_data
            
            ax = axes[j][i-2]
            
            plotFig(ax,csd,breaks,absmax)
    

        breaks = originalbreaks
    
        # plt.tight_layout()
        # plt.show()
    


In [ ]:
from scipy import io
import quantities as pq
import matplotlib.colors as colors

originalbreaks = breaks

fig, axes = plt.subplots(3,5,figsize=(12,12))

for j in range(3):

    for i in np.arange(2,7):    

        if j == 0:
            lfp_data = byLayer_objective_sphere_full[i].T * 1E-3 * pq.V
            breaks = originalbreaks
            z_data = np.arange(0,51) * 1e-6 * pq.m
            absmax = 1e-3
        elif j == 1:
            lfp_data = byLayer_objective_sphere_half[i].T * 1E-3 * pq.V
            breaks = originalbreaks/2
            z_data = np.arange(0,51,2) * 1e-6 * pq.m
            absmax = 1e-2
        else:
            lfp_data = byLayer_objective_sphere_quarter[i].T * 1E-3 * pq.V
            breaks = originalbreaks/4
            z_data = np.arange(0,51,4) * 1e-6 * pq.m
            absmax = 5e-2
        
        diam = 20E-6 * pq.m                              # [m]
        h = 1E-6 * pq.m                                 # [m]
        sigma = 0.277 * pq.S / pq.m                         # [S/m] or [1/(ohm*m)]
        sigma_top = 0.277 * pq.S / pq.m                     # [S/m] or [1/(ohm*m)]
    
        # Input dictionaries for each method
    
        std_input = {
            'lfp' : lfp_data,
            'coord_electrode' : z_data,
            'sigma' : sigma,
            'f_type' : 'gaussian',
            'f_order' : (3, 1),
        }
        #Create the different CSD-method class instances. We use the class methods
        #get_csd() and filter_csd() below to get the raw and spatially filtered
        #versions of the current-source density estimates.
        csd_dict = dict(
            std_csd = StandardCSD(**std_input),
        )
    
        for method, csd_obj in csd_dict.items():
    
            #plot raw csd estimate
            csd = lfp_data
            
            ax = axes[j][i-2]
            
            plotFig(ax,csd,breaks,absmax)
    

        breaks = originalbreaks
    
        # plt.tight_layout()
        # plt.show()
    
